In [1]:
# /// script
# requires-python = ">=3.12"
# dependencies = [
#     "anywidget>=0.9.0",
#     "jupyter-scatter-scsketch>=0.21.0",
#     "llvmlite>=0.44.0",
#     "numpy>=1.26.0",
#     "pandas>=2.0.0",
#     "scanpy>=1.9.0",
#     "scipy>=1.11.0",
#     "ipywidgets>=8.0.0",
#     "matplotlib>=3.7.0",
#     "requests>=2.31.0",
#     "watchfiles>=0.20.0",
# ]
#
# [tool.uv.sources]
# scsketch = { path = ".", editable = true }
# ///

# scSketch

> **Quick Start**: `uvx scsketch demo` (no cloning required!)  
> Or from this repo: `uvx juv run demo.ipynb`

scSketch provides a custom UI for [Jupyter-Scatter](https://jupyter-scatter.dev) that implements [Directional Analysis from Colubri et al's Sciviewer](https://doi.org/10.1093/bioinformatics/btab689). Sciviewer's directional analysis helps you interpret patterns in embedding visualizations by identifying genes varying locally along any user-specified direction.

For this demo, we're using the PBMC 3k (processed) dataset, since it is small (~2,600 cells), publicly available, and explicitly contains the UMAP coordinates and cluster metadata.

## Load Data

Load your single-cell data using scanpy and prepare it for visualization.

In [1]:
# Here we download the sample PBMC 3k dataset from the cellxgene repository

import urllib.request
import os.path

# data_url = "https://raw.githubusercontent.com/chanzuckerberg/cellxgene/main/example-dataset/pbmc3k.h5ad"
# data_file = "1M_20260121.h5ad"
data_file = "pbmc3k.h5ad"

# if not os.path.exists(data_file):
#     print("Downloading PBMC 3K demo...")
#     urllib.request.urlretrieve(data_url, data_file)
#     print("Download complete.")

In [2]:
import scanpy as sc

adata = sc.read(data_file)
adata

/opt/homebrew/Caskroom/miniforge/base/envs/scsketch091525/lib/python3.13/site-packages/anndata/_io/h5ad.py:267: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  return AnnData(
/opt/homebrew/Caskroom/miniforge/base/envs/scsketch091525/lib/python3.13/site-packages/anndata/_io/h5ad.py:267: FutureWarning: Moving element from .uns['neighbors']['connectivities'] to .obsp['connectivities'].

This is where adjacency matrices should go now.
  return AnnData(


AnnData object with n_obs × n_vars = 2638 × 1838
    obs: 'n_genes', 'percent_mito', 'n_counts', 'louvain'
    var: 'n_cells'
    uns: 'draw_graph', 'louvain', 'louvain_colors', 'neighbors', 'pca', 'rank_genes_groups'
    obsm: 'X_pca', 'X_tsne', 'X_umap', 'X_draw_graph_fr'
    varm: 'PCs'
    obsp: 'distances', 'connectivities'

## Launch scSketch

Create and display the scSketch widget.

In [3]:
from scsketch import ScSketch

sketch = ScSketch(
    adata=adata,
    metadata_cols=["louvain"],   # Metadata columns that can be used to color the cells
    color_by_default="louvain",  # A metadata column to use by default for coloring
    height=720,
    background_color="#111111",
)

sketch.show()

In [29]:
sketch.get_genes("Selection 1") # Get the list of selected genes as a dataframe for further analysis

,gene,correlation,p-value
7902,ENSG00000110700,0.630128,0.0
3820,ENSG00000145425,0.611698,0.0
2611,ENSG00000144713,0.586820,0.0
6955,ENSG00000156482,0.571171,0.0
78,ENSG00000116251,0.567364,0.0
...,...,...,...
4020,ENSG00000145649,-0.681225,0.0
10364,ENSG00000100450,-0.699937,0.0
13387,ENSG00000077984,-0.733709,0.0
12421,ENSG00000271503,-0.755271,0.0


## How to Use scSketch

1. **Select points**: Use the rectangle or lasso tool to select cells in the embedding
2. **Add selection**: Click the `+` button to save your selection
3. **Run analysis**: Click `Compute Directional Search` to identify genes varying along the selected direction
4. **Explore results**: Click on genes to see their Reactome pathways, and click pathways to view diagrams
5. **Color by genes**: Use the dropdown to color the embedding by specific genes or metadata
6. **Get list genes**: Use the get_genes() function in the scSketch object to retrieve the list of genes in a selected direction.

The directional analysis shows genes with their Pearson Correlation Coefficient (R) and p-value (p), representing which genes are most upregulated or downregulated along the selected direction.